In [1]:
%matplotlib inline
import xarray as xr
import numpy as np
from time import time
import pickle
import statsmodels.api as sm
from sklearn.linear_model import TheilSenRegressor, LinearRegression
import pymannkendall as mk
from scipy.stats import pearsonr

import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch
plt.rcParams['font.family'] = 'Myriad Pro'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300



path_ = '/home/mizu_home/xp53/nas/home/BEST/TAVG/'
pre_ = 'Complete_TAVG_Daily_LatLong1_'

from ipcc_colormap import *
cmap_prep = ipcc_cmap()
cmap_prep.read_rgb_data_from_excel()
;

''

In [2]:
# i concatenated two chunks of codes here and you may find duplicate variables (t_50 and temp_median)
# but im too lazy to clean it up so bear with me
# read processed data from pkl file
pkl_file = open('NH_winter_temp.pkl', 'rb')
# remove the first 90 rows (JFM of 1980) and the last 61 rows (ND of 2020)
temp = pickle.load(pkl_file)[90:-61, :, :]
climatology = np.mean(temp, axis=0)

temp_mean = np.zeros((39, 90, 360))
temp_median = np.zeros((39, 90, 360))
t_50 = np.zeros((39, 90, 360))
t_95 = np.zeros((39, 90, 360))
t_5 = np.zeros((39, 90, 360))

for yy in range(39):
    left, right = yy*151, (yy+1)*151

    tmean = np.mean(temp[left:right, :, :], axis=0)
    tmedian = np.median(temp[left:right, :, :], axis=0)
    temp_mean[yy, :, :] = tmean
    temp_median[yy, :, :] = tmedian

    t50 = np.percentile(temp[left:right, :, :], 50, axis=0)
    t95 = np.percentile(temp[left:right, :, :], 95, axis=0)
    t5 = np.percentile(temp[left:right, :, :], 5, axis=0)
    t_50[yy, :, :] = t50
    t_95[yy, :, :] = t95
    t_5[yy, :, :] = t5

# mask out nan values in temperatures
mask = 1 - np.isnan(np.mean(temp_mean, axis=0))
# the 1-deg mask (mask2) is downscaled from the 2-deg from Gottlieb 2024 et al.
mask2 = np.loadtxt('mask_1deg.txt')
mask = mask * mask2

In [6]:
# temp has shape (5889, 90, 360)
# 5889 days = 39 winters x 151 days/winter
# Reshape into (39, 151, 90, 360) format
temp_reshaped = np.zeros((39, 151, 90, 360))
for i in range(39):
    temp_reshaped[i] = temp[i*151:(i+1)*151, :, :]

# Conducting bootstrapping experiment for statistical significance of trend differences

N = 1000  # Number of bootstrap samples

# Initialize arrays to store bootstrapped trends
bootstrap_k_mean = np.zeros((N, 90, 360))
bootstrap_k_median = np.zeros((N, 90, 360))
bootstrap_k5 = np.zeros((N, 90, 360))
bootstrap_k95 = np.zeros((N, 90, 360))

# Initialize arrays for p-values
pval_mean_median = np.zeros((90, 360))
pval_5th_median = np.zeros((90, 360))
pval_95th_median = np.zeros((90, 360))

# Calculate trends on original data once
orig_k_mean = np.zeros((90, 360))
orig_k_median = np.zeros((90, 360))
orig_k5 = np.zeros((90, 360))
orig_k95 = np.zeros((90, 360))

xx = np.arange(39)  # years

print("Calculating trends for original data...")
for i in range(90):
    for j in range(360):
        if not mask[i, j]:
            continue
            
        # Mean trend
        lm1 = TheilSenRegressor(random_state=42)
        lm1.fit(xx[:,None], temp_mean[:,i,j])
        orig_k_mean[i,j] = lm1.coef_[0] * 10
        
        # Median trend
        lm2 = TheilSenRegressor(random_state=42)
        lm2.fit(xx[:,None], temp_median[:,i,j])
        orig_k_median[i,j] = lm2.coef_[0] * 10
        
        # 5th percentile trend
        lm5 = TheilSenRegressor(random_state=42)
        lm5.fit(xx[:,None], t_5[:,i,j])
        orig_k5[i,j] = lm5.coef_[0] * 10
        
        # 95th percentile trend
        lm95 = TheilSenRegressor(random_state=42)
        lm95.fit(xx[:,None], t_95[:,i,j])
        orig_k95[i,j] = lm95.coef_[0] * 10

# Original differences
orig_diff_mean_median = orig_k_mean - orig_k_median
orig_diff_5th_median = orig_k5 - orig_k_median
orig_diff_95th_median = orig_k95 - orig_k_median

print("Starting bootstrap procedure...")
t_start = time()

# Generate N bootstrap samples
for n in range(N):
    if (n+1) % 50 == 0:
        print(f"Bootstrap sample {n+1}/{N}, time elapsed: {time() - t_start:.2f}s")
    
    # Create bootstrapped dataset by resampling years with replacement
    year_indices = np.random.choice(39, size=39, replace=True)
    
    # Create bootstrapped temperature arrays
    boot_temp_mean = np.zeros((39, 90, 360))
    boot_temp_median = np.zeros((39, 90, 360))
    boot_t5 = np.zeros((39, 90, 360))
    boot_t95 = np.zeros((39, 90, 360))
    
    # For each sampled year, calculate statistics
    for i, year_idx in enumerate(year_indices):
        # Use the original year's data to preserve seasonality and spatial correlations
        boot_temp = temp_reshaped[year_idx]
        
        # Calculate statistics for this bootstrapped year
        boot_temp_mean[i] = np.mean(boot_temp, axis=0)
        boot_temp_median[i] = np.median(boot_temp, axis=0)
        boot_t5[i] = np.percentile(boot_temp, 5, axis=0)
        boot_t95[i] = np.percentile(boot_temp, 95, axis=0)
    
    # Calculate trends for each grid point
    for i in range(90):
        for j in range(360):
            if not mask[i, j]:
                continue
            
            # Calculate trends using TheilSenRegressor
            # Mean trend
            lm1 = TheilSenRegressor(random_state=42)
            lm1.fit(xx[:,None], boot_temp_mean[:,i,j])
            bootstrap_k_mean[n,i,j] = lm1.coef_[0] * 10
            
            # Median trend
            lm2 = TheilSenRegressor(random_state=42)
            lm2.fit(xx[:,None], boot_temp_median[:,i,j])
            bootstrap_k_median[n,i,j] = lm2.coef_[0] * 10
            
            # 5th percentile trend
            lm5 = TheilSenRegressor(random_state=42)
            lm5.fit(xx[:,None], boot_t5[:,i,j])
            bootstrap_k5[n,i,j] = lm5.coef_[0] * 10
            
            # 95th percentile trend
            lm95 = TheilSenRegressor(random_state=42)
            lm95.fit(xx[:,None], boot_t95[:,i,j])
            bootstrap_k95[n,i,j] = lm95.coef_[0] * 10

print(f"Bootstrap sampling completed in {time() - t_start:.2f}s")

# Calculate differences for each bootstrap sample
bootstrap_diff_mean_median = bootstrap_k_mean - bootstrap_k_median
bootstrap_diff_5th_median = bootstrap_k5 - bootstrap_k_median
bootstrap_diff_95th_median = bootstrap_k95 - bootstrap_k_median

# Calculate empirical p-values for each grid point
print("Calculating empirical p-values...")
for i in range(90):
    for j in range(360):
        if not mask[i, j]:
            continue
        
        # Mean - Median difference
        # Two-tailed test: count how many bootstrap samples have abs difference >= abs original difference
        abs_diff_orig = np.abs(orig_diff_mean_median[i,j])
        abs_diff_boot = np.abs(bootstrap_diff_mean_median[:,i,j])
        pval_mean_median[i,j] = np.mean(abs_diff_boot >= abs_diff_orig)
        
        # 5th percentile - Median difference
        abs_diff_orig = np.abs(orig_diff_5th_median[i,j])
        abs_diff_boot = np.abs(bootstrap_diff_5th_median[:,i,j])
        pval_5th_median[i,j] = np.mean(abs_diff_boot >= abs_diff_orig)
        
        # 95th percentile - Median difference
        abs_diff_orig = np.abs(orig_diff_95th_median[i,j])
        abs_diff_boot = np.abs(bootstrap_diff_95th_median[:,i,j])
        pval_95th_median[i,j] = np.mean(abs_diff_boot >= abs_diff_orig)

print("Bootstrapping analysis completed!")

# Save results
bootstrap_results = {
    'orig_k_mean': orig_k_mean,
    'orig_k_median': orig_k_median,
    'orig_k5': orig_k5,
    'orig_k95': orig_k95,
    'orig_diff_mean_median': orig_diff_mean_median,
    'orig_diff_5th_median': orig_diff_5th_median,
    'orig_diff_95th_median': orig_diff_95th_median,
    'pval_mean_median': pval_mean_median,
    'pval_5th_median': pval_5th_median,
    'pval_95th_median': pval_95th_median
}

# Optional: save bootstrap results to file
with open('bootstrap_results.pkl', 'wb') as f:
    pickle.dump(bootstrap_results, f)


Calculating trends for original data...
Starting bootstrap procedure...


KeyboardInterrupt: 